# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fc5eef78fa0>
├── 'a' --> tensor([[-0.7869, -0.2149,  0.7862],
│                   [-0.3194,  0.0272,  0.3979]])
└── 'x' --> <FastTreeValue 0x7fc5eef78ee0>
    └── 'c' --> tensor([[-0.7819, -0.6215,  0.8808,  0.5478],
                        [ 1.6267,  0.2569, -1.7004,  0.6652],
                        [ 0.0839, -0.3138,  0.2211,  2.4923]])

In [4]:
t.a

tensor([[-0.7869, -0.2149,  0.7862],
        [-0.3194,  0.0272,  0.3979]])

In [5]:
%timeit t.a

64.1 ns ± 0.0539 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fc5eef78fa0>
├── 'a' --> tensor([[ 0.4074,  0.5257, -0.5364],
│                   [-0.5351,  0.4156,  0.3310]])
└── 'x' --> <FastTreeValue 0x7fc5eef78ee0>
    └── 'c' --> tensor([[-0.7819, -0.6215,  0.8808,  0.5478],
                        [ 1.6267,  0.2569, -1.7004,  0.6652],
                        [ 0.0839, -0.3138,  0.2211,  2.4923]])

In [7]:
%timeit t.a = new_value

69.1 ns ± 0.0928 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.7869, -0.2149,  0.7862],
               [-0.3194,  0.0272,  0.3979]]),
    x: Batch(
           c: tensor([[-0.7819, -0.6215,  0.8808,  0.5478],
                      [ 1.6267,  0.2569, -1.7004,  0.6652],
                      [ 0.0839, -0.3138,  0.2211,  2.4923]]),
       ),
)

In [10]:
b.a

tensor([[-0.7869, -0.2149,  0.7862],
        [-0.3194,  0.0272,  0.3979]])

In [11]:
%timeit b.a

52.7 ns ± 0.0304 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.7166,  0.7516, -1.0538],
               [-0.2873,  0.1891,  0.7373]]),
    x: Batch(
           c: tensor([[-0.7819, -0.6215,  0.8808,  0.5478],
                      [ 1.6267,  0.2569, -1.7004,  0.6652],
                      [ 0.0839, -0.3138,  0.2211,  2.4923]]),
       ),
)

In [13]:
%timeit b.a = new_value

477 ns ± 0.144 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

825 ns ± 0.834 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 52 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 780 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

138 µs ± 545 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fc5170cb250>
├── 'a' --> tensor([[[-0.7869, -0.2149,  0.7862],
│                    [-0.3194,  0.0272,  0.3979]],
│           
│                   [[-0.7869, -0.2149,  0.7862],
│                    [-0.3194,  0.0272,  0.3979]],
│           
│                   [[-0.7869, -0.2149,  0.7862],
│                    [-0.3194,  0.0272,  0.3979]],
│           
│                   [[-0.7869, -0.2149,  0.7862],
│                    [-0.3194,  0.0272,  0.3979]],
│           
│                   [[-0.7869, -0.2149,  0.7862],
│                    [-0.3194,  0.0272,  0.3979]],
│           
│                   [[-0.7869, -0.2149,  0.7862],
│                    [-0.3194,  0.0272,  0.3979]],
│           
│                   [[-0.7869, -0.2149,  0.7862],
│                    [-0.3194,  0.0272,  0.3979]],
│           
│                   [[-0.7869, -0.2149,  0.7862],
│                    [-0.3194,  0.0272,  0.3979]]])
└── 'x' --> <FastTreeValue 0x7fc5170cbb20>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.4 µs ± 98.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fc5eef642e0>
├── 'a' --> tensor([[-0.7869, -0.2149,  0.7862],
│                   [-0.3194,  0.0272,  0.3979],
│                   [-0.7869, -0.2149,  0.7862],
│                   [-0.3194,  0.0272,  0.3979],
│                   [-0.7869, -0.2149,  0.7862],
│                   [-0.3194,  0.0272,  0.3979],
│                   [-0.7869, -0.2149,  0.7862],
│                   [-0.3194,  0.0272,  0.3979],
│                   [-0.7869, -0.2149,  0.7862],
│                   [-0.3194,  0.0272,  0.3979],
│                   [-0.7869, -0.2149,  0.7862],
│                   [-0.3194,  0.0272,  0.3979],
│                   [-0.7869, -0.2149,  0.7862],
│                   [-0.3194,  0.0272,  0.3979],
│                   [-0.7869, -0.2149,  0.7862],
│                   [-0.3194,  0.0272,  0.3979]])
└── 'x' --> <FastTreeValue 0x7fc517070f40>
    └── 'c' --> tensor([[-0.7819, -0.6215,  0.8808,  0.5478],
                        [ 1.6267,  0.2569, -1.7004,  0.6652],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 39.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

64.2 µs ± 87.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.7869, -0.2149,  0.7862],
                [-0.3194,  0.0272,  0.3979]],
       
               [[-0.7869, -0.2149,  0.7862],
                [-0.3194,  0.0272,  0.3979]],
       
               [[-0.7869, -0.2149,  0.7862],
                [-0.3194,  0.0272,  0.3979]],
       
               [[-0.7869, -0.2149,  0.7862],
                [-0.3194,  0.0272,  0.3979]],
       
               [[-0.7869, -0.2149,  0.7862],
                [-0.3194,  0.0272,  0.3979]],
       
               [[-0.7869, -0.2149,  0.7862],
                [-0.3194,  0.0272,  0.3979]],
       
               [[-0.7869, -0.2149,  0.7862],
                [-0.3194,  0.0272,  0.3979]],
       
               [[-0.7869, -0.2149,  0.7862],
                [-0.3194,  0.0272,  0.3979]]]),
    x: Batch(
           c: tensor([[[-0.7819, -0.6215,  0.8808,  0.5478],
                       [ 1.6267,  0.2569, -1.7004,  0.6652],
                       [ 0.0839, -0.3138,  0.2211,  2.4923]],
         

In [26]:
%timeit Batch.stack(batches)

79.7 µs ± 201 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.7869, -0.2149,  0.7862],
               [-0.3194,  0.0272,  0.3979],
               [-0.7869, -0.2149,  0.7862],
               [-0.3194,  0.0272,  0.3979],
               [-0.7869, -0.2149,  0.7862],
               [-0.3194,  0.0272,  0.3979],
               [-0.7869, -0.2149,  0.7862],
               [-0.3194,  0.0272,  0.3979],
               [-0.7869, -0.2149,  0.7862],
               [-0.3194,  0.0272,  0.3979],
               [-0.7869, -0.2149,  0.7862],
               [-0.3194,  0.0272,  0.3979],
               [-0.7869, -0.2149,  0.7862],
               [-0.3194,  0.0272,  0.3979],
               [-0.7869, -0.2149,  0.7862],
               [-0.3194,  0.0272,  0.3979]]),
    x: Batch(
           c: tensor([[-0.7819, -0.6215,  0.8808,  0.5478],
                      [ 1.6267,  0.2569, -1.7004,  0.6652],
                      [ 0.0839, -0.3138,  0.2211,  2.4923],
                      [-0.7819, -0.6215,  0.8808,  0.5478],
                      [ 1.6267,  

In [28]:
%timeit Batch.cat(batches)

145 µs ± 200 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

315 µs ± 1.63 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
